### **HuggingFace Login**

In [1]:
from huggingface_hub import login
login()

### **Import Libairies**

In [2]:
from IPython.display import display

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import Dataset



### **Read Datasets**

In [3]:
'''
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)

df_train = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/train.csv", index_col=0)
df_test = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/test.csv", index_col=0)
'''

df_train = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/train.csv", index_col=0)
df_test = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/test.csv", index_col=0)


### **Short Analysis**

In [4]:
print(f"Length - train {len(df_train)} - test {len(df_test)}")

Length - train 7613 - test 3263


### **Preprocess Datasets**

##### _Merge columns (full)_

In [5]:
keywords = pd.concat([
    pd.DataFrame(df_train["keyword"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["keyword"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
keywords.head()

,train,test
keyword,,
fatalities,45,5
deluge,42,8
armageddon,42,8
damage,41,9
body%20bags,41,9


In [6]:
locations = pd.concat([
    pd.DataFrame(df_train["location"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["location"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
locations.head()

,train,test
location,,
USA,104.0,37.0
New York,71.0,38.0
United States,50.0,15.0
London,45.0,13.0
Canada,29.0,13.0


In [7]:
df_train_full = df_train.copy()
df_train_full['text'] = df_train_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)
df_test_full = df_test.copy()
df_test_full['text'] = df_test_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)


### **Model Assessment**

In [8]:
def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Value': [f1, precision, recall]
    }, index=['F1 Score', 'Precision', 'Recall'])
    
    cm_df = pd.DataFrame(cm, columns=['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])
    
    return metrics_df, cm_df

### **DISTIL BERT Tokenizer**

##### _DISTIL BERT Tokenizer + split validation_

In [10]:
distill_bert_tokenizer_full = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

distill_bert_encodings_full = distill_bert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
distill_bert_labels_full = torch.tensor(list(df_train_full['target']))

distill_bert_input_ids_train_full, distill_bert_input_ids_valid_full, \
distill_bert_attention_mask_train_full, distill_bert_attention_mask_valid_full, distill_bert_y_train_full, distill_bert_y_valid_full = train_test_split(
    distill_bert_encodings_full['input_ids'], 
    distill_bert_encodings_full['attention_mask'], 
    distill_bert_labels_full, 
    test_size=0.15, 
    stratify=distill_bert_labels_full, 
    random_state=0
)

distill_bert_train_encodings_full = {
    'input_ids': torch.tensor(distill_bert_input_ids_train_full),
    'attention_mask': torch.tensor(distill_bert_attention_mask_train_full)
}

distill_bert_valid_encodings_full = {
    'input_ids': torch.tensor(distill_bert_input_ids_valid_full),
    'attention_mask': torch.tensor(distill_bert_attention_mask_valid_full)
}

distill_bert_train_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_train_encodings_full['input_ids'],
    "attention_mask": distill_bert_train_encodings_full['attention_mask'],
    "labels": distill_bert_y_train_full
})

distill_bert_valid_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_valid_encodings_full['input_ids'],
    "attention_mask": distill_bert_valid_encodings_full['attention_mask'],
    "labels": distill_bert_y_valid_full
})

distill_bert_test_encodings_full = distill_bert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)

distill_bert_test_encodings_full = {
    key: torch.tensor(val) for key, val in distill_bert_test_encodings_full.items()
    }

distill_bert_test_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_test_encodings_full['input_ids'],
    "attention_mask": distill_bert_test_encodings_full['attention_mask']
})

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\cauchepy\Git\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

##### _Save HuggingFace DISTIL BERT Tokenizer_

In [11]:
distill_bert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_tokenizer")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\cauchepy\Git\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\models--yanncauchepin--kaggle_disastertweets_distill_bert_tokenizer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_disastertweets_distill_bert_tokenizer/commit/e2566ad7feb72933925f6db6828bef8465007d47', commit_message='Upload tokenizer', commit_description='', oid='e2566ad7feb72933925f6db6828bef8465007d47', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_disastertweets_distill_bert_tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_disastertweets_distill_bert_tokenizer'), pr_revision=None, pr_num=None)

### **DISTIL BERT Training from scratch**

##### _DISTIL BERT Transformers + local save_

In [13]:
distill_bert_model_full = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    hub_model_id="kaggle_disastertweets_distill_bert_model",
    output_dir="./distil_bert_results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./distil_bert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

distill_bert_trainer_full = Trainer(
    model=distill_bert_model_full,
    args=training_args,
    train_dataset=distill_bert_train_dataset_full,
    eval_dataset=distill_bert_valid_dataset_full
)

distill_bert_trainer_full.train()

distill_bert_trainer_full.save_model("distil_bert_model") # Local

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/24270 [00:00<?, ?it/s]

{'loss': 0.6997, 'grad_norm': 2.6173481941223145, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.7122, 'grad_norm': 2.223590612411499, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.7147, 'grad_norm': 4.8948893547058105, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.692, 'grad_norm': 2.732520341873169, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 0.6894, 'grad_norm': 3.0921616554260254, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.6848, 'grad_norm': 2.9021666049957275, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.6751, 'grad_norm': 2.748015880584717, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 0.6954, 'grad_norm': 1.9925870895385742, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6526, 'grad_norm': 3.4700844287872314, 'learning_rate': 9e-06, 'epoch': 0.06}
{'loss': 0.6595, 'grad_norm': 3.2392842769622803, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.6438, 'grad_norm': 2.

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 0.42939692735671997, 'eval_runtime': 3.1107, 'eval_samples_per_second': 367.115, 'eval_steps_per_second': 91.939, 'epoch': 1.0}
{'loss': 0.2882, 'grad_norm': 5.9888224601745605, 'learning_rate': 4.764408918805217e-05, 'epoch': 1.0}
{'loss': 0.4133, 'grad_norm': 8.687431335449219, 'learning_rate': 4.762305427008834e-05, 'epoch': 1.01}
{'loss': 0.5885, 'grad_norm': 0.9078088998794556, 'learning_rate': 4.7602019352124527e-05, 'epoch': 1.01}
{'loss': 0.3999, 'grad_norm': 0.5753321647644043, 'learning_rate': 4.758098443416071e-05, 'epoch': 1.02}
{'loss': 0.4255, 'grad_norm': 8.060151100158691, 'learning_rate': 4.755994951619689e-05, 'epoch': 1.03}
{'loss': 0.2921, 'grad_norm': 11.504206657409668, 'learning_rate': 4.753891459823307e-05, 'epoch': 1.03}
{'loss': 1.015, 'grad_norm': 18.695791244506836, 'learning_rate': 4.7517879680269253e-05, 'epoch': 1.04}
{'loss': 0.5828, 'grad_norm': 5.514754772186279, 'learning_rate': 4.749684476230543e-05, 'epoch': 1.04}
{'loss': 0.4135, 'gra

...

{'eval_loss': 1.7277015447616577, 'eval_runtime': 2.9178, 'eval_samples_per_second': 391.394, 'eval_steps_per_second': 98.02, 'epoch': 13.0}
{'loss': 0.0003, 'grad_norm': 0.0054154302924871445, 'learning_rate': 6.7942785023138414e-06, 'epoch': 13.0}
{'loss': 0.0002, 'grad_norm': 0.0061158351600170135, 'learning_rate': 6.7732435843500215e-06, 'epoch': 13.01}
{'loss': 0.0002, 'grad_norm': 0.05792057514190674, 'learning_rate': 6.7522086663862015e-06, 'epoch': 13.02}
{'loss': 0.0011, 'grad_norm': 0.004359594080597162, 'learning_rate': 6.7311737484223815e-06, 'epoch': 13.02}
{'loss': 0.0002, 'grad_norm': 0.0035768544767051935, 'learning_rate': 6.7101388304585624e-06, 'epoch': 13.03}
{'loss': 0.0002, 'grad_norm': 0.0028347016777843237, 'learning_rate': 6.689103912494742e-06, 'epoch': 13.03}
{'loss': 0.0002, 'grad_norm': 0.004171994514763355, 'learning_rate': 6.668068994530921e-06, 'epoch': 13.04}
{'loss': 0.2032, 'grad_norm': 0.012397993355989456, 'learning_rate': 6.647034076567102e-06, 'epo

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 1.6480815410614014, 'eval_runtime': 2.9456, 'eval_samples_per_second': 387.694, 'eval_steps_per_second': 97.093, 'epoch': 14.0}
{'loss': 0.0007, 'grad_norm': 0.004899166524410248, 'learning_rate': 3.3866217921750107e-06, 'epoch': 14.0}
{'loss': 0.0002, 'grad_norm': 0.005032229702919722, 'learning_rate': 3.3655868742111908e-06, 'epoch': 14.01}
{'loss': 0.0002, 'grad_norm': 0.0073255738243460655, 'learning_rate': 3.344551956247371e-06, 'epoch': 14.02}
{'loss': 0.0391, 'grad_norm': 0.005732337478548288, 'learning_rate': 3.323517038283551e-06, 'epoch': 14.02}
{'loss': 0.0002, 'grad_norm': 0.0071000102907419205, 'learning_rate': 3.302482120319731e-06, 'epoch': 14.03}
{'loss': 0.0001, 'grad_norm': 0.006009625270962715, 'learning_rate': 3.2814472023559113e-06, 'epoch': 14.04}
{'loss': 0.0002, 'grad_norm': 0.013703993521630764, 'learning_rate': 3.2604122843920914e-06, 'epoch': 14.04}
{'loss': 0.1274, 'grad_norm': 0.0063869948498904705, 'learning_rate': 3.2393773664282706e-06, 'ep

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 1.7922857999801636, 'eval_runtime': 3.0903, 'eval_samples_per_second': 369.539, 'eval_steps_per_second': 92.546, 'epoch': 15.0}
{'train_runtime': 1613.2676, 'train_samples_per_second': 60.167, 'train_steps_per_second': 15.044, 'train_loss': 0.17204596451727097, 'epoch': 15.0}


##### _Save HugggingFace trained DISTIL BERT Transformers_ 

In [14]:
distill_bert_trainer_full.push_to_hub()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_disastertweets_distill_bert_model/commit/03cd754de4f7cae1bbd4d45928a82dfed6c99e8f', commit_message='End of training', commit_description='', oid='03cd754de4f7cae1bbd4d45928a82dfed6c99e8f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_disastertweets_distill_bert_model', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_disastertweets_distill_bert_model'), pr_revision=None, pr_num=None)

### **DISTIL BERT Loading from pretrained**

In [ ]:
distill_bert_model_full = DistilBertForSequenceClassification.from_pretrained("distil_bert_model")

training_args = TrainingArguments(
    hub_model_id="kaggle_disastertweets_distill_bert_model",
    output_dir="./distil_bert_results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./distil_bert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

distill_bert_trainer_full = Trainer(
    model=distill_bert_model_full,
    args=training_args,
    train_dataset=distill_bert_train_dataset_full,
    eval_dataset=distill_bert_valid_dataset_full
)

### **DISTIL BERT Assessment**

In [15]:
distill_bert_predictions_full = distill_bert_trainer_full.predict(distill_bert_valid_dataset_full)
distill_bert_logits_full = distill_bert_predictions_full.predictions
distill_bert_y_pred_full = np.argmax(distill_bert_logits_full, axis=1)

distill_bert_trainer_full_assessement = evaluate_classifier(distill_bert_y_valid_full.numpy(), distill_bert_y_pred_full)
display(distill_bert_trainer_full_assessement[0])
display(distill_bert_trainer_full_assessement[1])

  0%|          | 0/286 [00:00<?, ?it/s]

,Value
F1 Score,0.822263
Precision,0.827804
Recall,0.824869


,Predicted Negative,Predicted Positive
Actual Negative,590,61
Actual Positive,139,352


### **DISTIL BERT Test Predictions**

##### _DISTIL BERT Test Predictions_

In [16]:
distill_bert_test_predictions_full = distill_bert_trainer_full.predict(distill_bert_test_dataset_full)
distill_bert_test_logits_full = distill_bert_test_predictions_full.predictions
distill_bert_test_y_pred_full = np.argmax(distill_bert_test_logits_full, axis=1)

distill_bert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': distill_bert_test_y_pred_full.flatten()
})

  0%|          | 0/816 [00:00<?, ?it/s]

##### _Save HuggingFace DISTIL BERT Test Predictions_

In [17]:
hf_distill_bert_test_submission_full = Dataset.from_pandas(distill_bert_test_submission_full)
hf_distill_bert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_submission_df")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/299 [00:00<?, ?B/s]

c:\Users\cauchepy\Git\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\datasets--yanncauchepin--kaggle_disastertweets_distill_bert_submission_df. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_distill_bert_submission_df/commit/978d80360b523a4ec4270c0d3c54dd917d1e09db', commit_message='Upload dataset', commit_description='', oid='978d80360b523a4ec4270c0d3c54dd917d1e09db', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_distill_bert_submission_df', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yanncauchepin/kaggle_disastertweets_distill_bert_submission_df'), pr_revision=None, pr_num=None)